In [5]:
import os
import librosa
import numpy as np
import soundfile as sf

# Define paths
base_path = r"F:\Level 3 Term 2\DSP Project\Data\MS-SNSD-master\MS-SNSD-master"
datasets = {
    "test": {
        "merged_path": os.path.join(base_path, "noisy_clean_test"),
        "noise_path": os.path.join(base_path, "noise_test"),
        "output_path": os.path.join(base_path, "export_clean_test")
    },
    "train": {
        "merged_path": os.path.join(base_path, "noisy_clean_train"),
        "noise_path": os.path.join(base_path, "noise_train"),
        "output_path": os.path.join(base_path, "export_clean_train")
    }
}

# Ensure output directories exist
for dataset in datasets.values():
    os.makedirs(dataset["output_path"], exist_ok=True)

def spectral_subtraction(noisy_path, noise_path, output_path):
    # List audio files
    noisy_files = sorted([os.path.join(noisy_path, f) for f in os.listdir(noisy_path) if f.endswith('.wav')])
    noise_files = sorted([os.path.join(noise_path, f) for f in os.listdir(noise_path) if f.endswith('.wav')])

    if len(noisy_files) != len(noise_files):
        raise ValueError(f"Mismatch in number of files between {noisy_path} and {noise_path}.")

    for i, (noisy_file, noise_file) in enumerate(zip(noisy_files, noise_files)):
        # Load noisy and noise audio
        noisy_audio, sr = librosa.load(noisy_file, sr=None)
        noise_audio, _ = librosa.load(noise_file, sr=None)

        # Match the length of noise to noisy audio
        if len(noise_audio) > len(noisy_audio):
            noise_audio = noise_audio[:len(noisy_audio)]
        else:
            noise_audio = np.pad(noise_audio, (0, len(noisy_audio) - len(noise_audio)))

        # Compute STFTs
        noisy_stft = librosa.stft(noisy_audio)
        noise_stft = librosa.stft(noise_audio)

        noisy_mag, noisy_phase = librosa.magphase(noisy_stft)
        noise_mag, _ = librosa.magphase(noise_stft)

        # Ensure spectrogram shapes match
        min_frames = min(noisy_mag.shape[1], noise_mag.shape[1])
        noisy_mag = noisy_mag[:, :min_frames]
        noise_mag = noise_mag[:, :min_frames]

        # Spectral subtraction
        clean_mag = np.maximum(noisy_mag - noise_mag, 0)

        # Reconstruct STFT
        clean_stft = clean_mag * noisy_phase[:, :min_frames]
        clean_audio = librosa.istft(clean_stft)

        # Save the cleaned audio
        output_file = os.path.join(output_path, f"clean_{i + 1}.wav")
        sf.write(output_file, clean_audio, sr)

    print(f"Processing complete! Cleaned audios saved in '{output_path}'.")

# Process both test and train datasets
for dataset, paths in datasets.items():
    print(f"Processing {dataset} dataset...")
    spectral_subtraction(paths["merged_path"], paths["noise_path"], paths["output_path"])


Processing test dataset...
Processing complete! Cleaned audios saved in 'F:\Level 3 Term 2\DSP Project\Data\MS-SNSD-master\MS-SNSD-master\export_clean_test'.
Processing train dataset...
Processing complete! Cleaned audios saved in 'F:\Level 3 Term 2\DSP Project\Data\MS-SNSD-master\MS-SNSD-master\export_clean_train'.
